In [32]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from aif360.datasets import BinaryLabelDataset
from aif360.algorithms.preprocessing import Reweighing
from aif360.metrics import ClassificationMetric

In [34]:
# Load the data
train = pd.read_csv(r'../Resources/Data/baseline_train_data.csv')
test = pd.read_csv(r'../Resources/Data/baseline_test_data.csv')
valid = pd.read_csv(r'../Resources/Data/baseline_valis_data.csv')

# Define target and features
target = 'two_year_recid'

# Separate features and target for training, testing, and validation data
X_train = train.drop(columns=[target])
y_train = train[target]

X_test = test.drop(columns=[target])
y_test = test[target]

X_valid = valid.drop(columns=[target])
y_valid = valid[target]

In [36]:
# Apply reweighting to the training set
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

In [38]:
# Convert training data to AIF360 BinaryLabelDataset format
train_aif = BinaryLabelDataset(df=pd.concat([X_train, y_train], axis=1),
                               label_names=[target],
                               protected_attribute_names=['sex'])

reweigher = Reweighing(unprivileged_groups=unprivileged_groups,
                       privileged_groups=privileged_groups)
train_aif_rw = reweigher.fit_transform(train_aif)

In [40]:
# Fit the logistic regression model with sample weights
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(train_aif_rw.features, train_aif_rw.labels.ravel(), sample_weight=train_aif_rw.instance_weights)

# Predict on the test data
y_pred_test = log_reg.predict(X_test)

C:\Users\General\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [42]:
# Combine predicted labels with the test features
test_with_pred = X_test.copy()
test_with_pred[target] = y_test
test_with_pred['y_pred'] = y_pred_test

# Convert to AIF360 BinaryLabelDataset format
test_aif = BinaryLabelDataset(df=pd.concat([X_test, y_test], axis=1),
                              label_names=[target],
                              protected_attribute_names=['sex'])

test_aif_pred = BinaryLabelDataset(df=pd.concat([X_test, pd.Series(y_pred_test, name=target)], axis=1),
                                   label_names=[target],
                                   protected_attribute_names=['sex'])

In [46]:
# Compute fairness metrics
metric_test_pred = ClassificationMetric(test_aif, test_aif_pred,
                                        unprivileged_groups=unprivileged_groups,
                                        privileged_groups=privileged_groups)

stat_parity_diff = metric_test_pred.statistical_parity_difference()
avg_odds_diff = metric_test_pred.average_odds_difference()
equal_opp_diff = metric_test_pred.equal_opportunity_difference()

# Display results
print(f"Statistical Parity Difference: {stat_parity_diff}")
print(f"Average Odds Difference: {avg_odds_diff}")
print(f"Equal Opportunity Difference: {equal_opp_diff}")

Statistical Parity Difference: 0.05941337233919708
Average Odds Difference: 0.049523719720250076
Equal Opportunity Difference: 0.08525594808940151
